<a href="https://colab.research.google.com/github/laurenneal/capstone-visual-neuroscience/blob/main/Feature_Engineering_from_Raw_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import sys
from numpy.ma.core import ceil, floor
from more_itertools import sliced

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# FUNCTIONS (Just for setting up functions, not running code)
This section contains functions that can be used on datasets for feature engineering

## Train Test Split on Stacks

In [2]:
# def trainTestSplit(df, splt = [.7,.2,.1]):
#   # splits on roi, rois accross stacks will have the same group (important if stacks are from the same video)
#   # split is the train test valuidation split (proportions given in that order)
#   if sum(splt) < 0.9999 or sum(splt) > 1.0001:
#     sys.exit('Splt must add to 1')
  
#   num_rois = len(df.index.unique(level='roi'))

#   tr_ind = int(ceil(num_rois*splt[0]))
#   ts_ind = int(ceil(num_rois*splt[1])) + tr_ind

#   ar = np.arange(num_rois)
#   np.random.shuffle(ar)
#   ar = ar+1 # roi id's start from 1

#   train = ar[:tr_ind]
#   test = ar[tr_ind:ts_ind]
#   val = ar[ts_ind:]

#   for i in train:
#     df.loc[pd.IndexSlice[:, :, i],'training'] = 'train'

#   for i in test:
#     df.loc[pd.IndexSlice[:, :, i],'training'] = 'test'

#   for i in val:
#     df.loc[pd.IndexSlice[:, :, i],'training'] = 'validate'

#   return(df)

## Combine Frames into Temporal Chunks - NOT Working

In [3]:
# # select frame size
# temporal_period_length = 200

In [4]:

# def groupFrames(df, period):
#   # inputs are data frame and temporal period length
#   # df needs to be sorted correctly, the indexing should take care of that

#   # check that all rois are the same size
#   # if this is a problem we can write a more computationally heavy workthrough
#   if (raw_df.groupby(['stack','roi']).count()['filename'].max() != raw_df.groupby(['stack','roi']).count()['filename'].min()):
#     sys.exit('ROIs need to be the same length')

#   num_stacks = len(df.index.unique(level='stack'))
#   num_roi = len(df.index.unique(level='roi'))

#   # get first label
#   lbl_1 = raw_df.head(1).index.values[0][1]
#   # use it to find the number of frames in a roi
#   num_frames = raw_df.loc[(1,lbl_1,1),'filename'].count()
#   # how many groups will ther be in each roi?
#   num_pds = int(floor(num_frames/period))
#   # how many left over
#   remainder_pds = num_frames%period

#   # iterate a column for one roi
#   lst = [[li + 1] * period for li in range(0,num_pds)]
#   lst = [li for sublist in lst for li in sublist] # https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-a-list-of-lists
#   lst = lst + [lst[-1] + 1] * remainder_pds
#   lst

#   new_col = np.array(lst * num_stacks * num_roi)
  
#   df['frame_group'] = new_col

#   return(df)

In [5]:
def groupFrames(df_in, period):
  
  # inputs are data frame and number of periods per stack
  df = df_in.copy()

  df = df.reset_index()
  # set a unique id for each roi stack combo
  df['long_id'] = df['movie_ID'] + '+' + df['stack'].astype('str') + '+' + df['roi'].astype('str')
  df = df.set_index('long_id')
  long_ids = df.index.unique()

  # for each unique roi we calculate a grouping index
  new_col = []
  for id in long_ids:
    # number of rows for each unique id
    num_rows = df.loc[id].shape[0]
    
    # how many frames per division to split the roi into number of periods
    frm_pd = int(floor(num_rows/period))

    # iterate a column for one roi
    lst = [[li + 1] * frm_pd for li in range(0,period)]
    lst = [li for sublist in lst for li in sublist] # https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-a-list-of-lists

    if len(lst) > num_rows:
      lst = lst[:num_rows] # remove some on the back if needed
    elif len(lst) < num_rows:
      lst = lst + [lst[-1]] * (num_rows - len(lst)) # add a few extra on the end if needed

    # checks to make sure the list has the correct number of rows
    if len(lst) != num_rows:
      sys.exit('Error 1: Somethings wrong with this code')

    new_col += lst

  # new col length should match the column length
  if len(new_col) != df.shape[0]:
    sys.exit('Error 2: Somethings wrong with this code')

  df['frame_group'] = new_col

  df = df.reset_index().drop(columns = 'long_id')

  return(df)

## Stim 1 feature engineering

For 16-len vector

In [6]:
from operator import setitem
def stimResponse(df, stim, response, zero_point = 0):
  """
  Takes a df, the name of the col holding the stimulus, the name of the col holding the response, and the value to use as the zero point

  Returns the df with the relationship type and the relation value added

  A different function will calculate the relationship between time periods
  """


  # takes a stim column and response column and returns two new feature column
  # PR is Positive Response
  # NR is Negative Response
  # PS is Positive Stimulus
  # NS is Negative Stimulus
  df_in = df.copy()
  #
  PR = df_in[response] > zero_point
  NR = df_in[response] <= zero_point
  PS = df_in[stim] > zero_point
  NS = df_in[stim] <= zero_point

  PR = PR.rename('PR')
  NR = NR.rename('NR')
  PS = PS.rename('PS')
  NS = NS.rename('NS')

  # combine into df
  df = pd.concat([df_in[stim], df_in[response], PR,NR,PS,NS], axis=1)
  #set the data types back to boolean
  df = df.astype({'PR': bool,
                'PS': bool,
                'NR': bool,
                'PR': bool})

  # get min and max values for normalizing
  S_min = df[stim].min()
  S_max = df[stim].max()
  R_min = df[response].min()
  R_max = df[response].max()

  #add a column normalizing the stim and response
  df['stim_norm'] = df[stim].apply(lambda x: x/S_max if x>0 else x/S_min)
  df['resp_norm'] = df[response].apply(lambda x: x/R_max if x>0 else x/R_min)


  # multiply stim and resp
  df['relation'] = df['stim_norm'] * df['resp_norm']

  #Set a new column with the category each row's relationship falls under
  df.loc[(df['PR'] & df['PS']),'relation_type'] = 'PRPS'
  df.loc[(df['NR'] & df['NS']),'relation_type'] = 'NRNS'
  df.loc[(df['PR'] & df['NS']),'relation_type'] = 'PRNS'
  df.loc[(df['NR'] & df['PS']),'relation_type'] = 'NRPS'
  
  df_in['relation_type_' + stim] = df['relation_type']
  df_in['relation_' + stim] = df['relation']
  return(df_in)

In [7]:
def get_future_stim_and_relationship(df_in, stim, time_window): 
  """
  takes a df, the stimulus name we're comparing, and the time window we're looking to in the future (in frames)

  returns a new df with two extra columns containing the relationship type and the value at the future time period
  """

  #make copy of the df to avoid editing the one in memory
  df = df_in.copy()

  #procedurally generate the column names we need to reference
  current_relation_colname = 'relation_'+stim
  current_relation_type_colname = 'relation_type_'+stim

  #and the column names we're creating
  future_relation_colname = 'relation_'+stim+'_+'+str(time_window)
  future_relation_type_colname = 'relation_type_'+stim+'_+'+str(time_window)

  #reset the index so that we can join the new columns later
  df = df.reset_index()


  # for each of the new columns, do four steps

  #get the existing array of relation values or relation types

  #cut off the first x values, where x is the time window we want to look in the future (done by indexing [time_window:])
        #this will drop the time x values in the future to line up with the current time

  #pad the end of the array with zeros by the same number of values so that the lengths still match (done by concatenating with np.zeros(time_window))

  #set the new array as the new 'future' column
  df[future_relation_colname] = pd.concat([df[current_relation_colname] \
                                           .reset_index(drop=True)[time_window:], \
                                           pd.Series((np.zeros(time_window)))], ignore_index = True)
  
  #repeat for the relation type
  df[future_relation_type_colname] = pd.concat([df[current_relation_type_colname] \
                                           .reset_index(drop=True)[time_window:], \
                                           pd.Series((np.zeros(time_window)))], ignore_index = True)

  #need to stop the end of one stack from referencing the beginning of another

  #get the index of any row where the frame # is high enough that x-frames in the future would run into another stack
  #by checking that the frame number is greater than the max frame number - (time window-1) (corrects for index number)
  end_of_stack_index = df.loc[df['frame'] >= max(df['frame'])-(time_window)].index

  #use .loc to set the two new columns to None for those rows
  df.loc[end_of_stack_index, future_relation_colname] = None
  df.loc[end_of_stack_index, future_relation_type_colname] = None

  #set the index back to the way it was
  df = df.set_index(['stack','label','roi','frame'])

  return df

In [8]:
def current_future_relationship(df, stim, time_window):
  """
  Takes df with a column for stim and a column for stim + a given time window. Returns the relationship.
  """
  # column names for current row
  current_relation_colname = 'relation_'+stim
  current_relation_type_colname = 'relation_type_'+stim

  # column names for rows in the future
  future_relation_colname = 'relation_'+stim+'_+'+str(time_window)
  future_relation_type_colname = 'relation_type_'+stim+'_+'+str(time_window)

  #with this approach, we need to drop the last x frames from each stack, so dropna
  df = df.dropna()

  #make a new column with the combo of current relation type and future relation type
  df['full_relation_type'] = df[current_relation_type_colname] + '-' + df[future_relation_type_colname]

  #one-hot encode the overall relationship, then multiply the new columns by the product of the relation values
  df = pd.get_dummies(df, columns=['full_relation_type'], prefix = current_relation_colname)

  #get a list of the columns created
  dummy_cols = [col for col in df.columns if current_relation_colname in col]

  #multiply the dummy columns by the relation value to distribute the value to the appropriate column
  for col in dummy_cols:
    df[col] = df[col] * (df[current_relation_colname]*df[future_relation_colname])

  return df

In [9]:
#for ease of use, package all four functions along with the dropna into one function for use later

def contrast_16_len_representation(df_in, stim, response, time_window, zero_point = 0):
  """
  helper function to run all of the stim1 functions that create the 16-length representation

  takes the df, stim name, response column name, time period to look in the future, and the zero point to compare positive and negative
  """

  df = df_in.copy()

  #get the current stim and response relationship
  df = stimResponse(df, stim, response, zero_point)
  #get the future stim and response relationship
  df = get_future_stim_and_relationship(df, stim, time_window)
  #get the relationship b/w the current and future stim/response columns
  df = current_future_relationship(df, stim, time_window)

  return df

4-len representation (assumes we will drop all negative response

In [10]:
from operator import setitem
def contrast_4_len_representation(df, stim, response, time_window, zero_point = 0):
  """
  Takes a df, the name of the col holding the stimulus, the name of the col holding the response, and the value to use as the zero point

  Returns the df with four extra columns holding the normalized response * stim1 for the four possible relationships

  the sign of the response is ignored because all frames with negative response will be dropped
  """


  # takes a stim column and returns two new feature column
  # PS is Positive Stimulus
  # NS is Negative Stimulus
  df_in = df.copy()

  #
  PS = df_in[stim] > zero_point
  NS = df_in[stim] <= zero_point

  PS = PS.rename('PS')
  NS = NS.rename('NS')

  # combine into df
  df = pd.concat([df_in[stim], df_in[response], PS,NS], axis=1)
  #set the data types back to boolean
  df = df.astype({'NS': bool,
                  'PS': bool})

  # get min and max values for normalizing
  S_min = df[stim].min()
  S_max = df[stim].max()
  R_min = df[response].min()
  R_max = df[response].max()

  #add a column normalizing the stim and response
  df['stim_norm'] = df[stim].apply(lambda x: x/S_max if x>0 else x/S_min)
  df['resp_norm'] = df[response].apply(lambda x: x/R_max if x>0 else x/R_min)


  # multiply stim and resp
  df['relation'] = df['stim_norm'] * df['resp_norm']

  #Set a new column with the category each row's relationship falls under
  df.loc[(df['PS']),'relation_type'] = 'PS'
  df.loc[(df['NS']),'relation_type'] = 'NS'
  
  df_in['relation_type_' + stim] = df['relation_type']
  df_in['relation_' + stim] = df['relation']

  df_in = get_future_stim_and_relationship(df_in, stim, time_window)
  df_in = df_in.dropna()

  df_in = current_future_relationship(df_in, stim, time_window)

  return(df_in)

# Stim 2, 3, and 5 feature engineering

In [11]:
# def directional_stim_categories(df, stim, response):
#   """
#   DEPRECATED - this was for when the direction was a single attribute of the degree of motion


#   Takes a df, the name of the col holding the direction of motion stimululi, the name of the col holding the response
#   Returns the df with the stim broken out into five cols based on direction of motion

#   A different function will calculate the relationship between response and direction
#   """


#   # add a column that categorizes the direction into four quadrants (segments can be changed)
#   df_in = df.copy()

#   #set conditions checking the direction of the stimulus - there has to be a better way but this is it for now
#   conditions = [
#     ((-45 <= df_in[stim]) & (df_in[stim] < 0)), #northwest
#     ((0 < df_in[stim]) & (df_in[stim] < 45)), #northeast
#     ((45 <= df_in[stim]) & (df_in[stim] < 135)), #east
#     ((135 <= df_in[stim]) & (df_in[stim] <= 180)),#southeast
#     ((-180 <= df_in[stim]) & (df_in[stim] < -135)), #southwest
#     ((-135 <= df_in[stim]) & (df_in[stim] < -45)), #west
#     (df_in[stim] == 0) #no movement
#     ]

#   #set the names we assign to those conditions
#   values = ['up', 'up', 'right', 'down', 'down', 'left', 'no_motion']

#   #add a column to the df holding the direction category
#   df['direction'] = np.select(conditions, values)

#   #one-hot encode the quadrants
#   df = pd.get_dummies(df, columns=['direction'], prefix = 'direction')

#   #get the names of the 4 direction columns we just added
#   direction_cols = [col for col in df.columns if 'direction' in col]


#   return(df)

In [12]:
def direction_orientation_stim_features(df, stim_x, stim_y, stim_orientation, response):
  """
  DEPRECATED - this was for when the direction was a single attribute of the degree of motion


  Takes a df, the name of the col holding the direction of motion stimululi (x and y), the name of the col holding the response
  Returns the df with two three extra columns holding the resp*stim for each feature

  """


  df_in = df.copy()


  #multiply the response by the three directional features
  df['stim2_feature'] = df[response] * df[stim_x]

  df['stim3_feature'] = df[response] * df[stim_y]

  df['stim5_feature'] = df[response] * df[stim_orientation]


  return(df)

# Stim 4, 7, and 8 feature engineering

In [13]:
def speed_freq_Response(df, stim_speed, stim_spatial, stim_temp, response):
  """
  Takes a df, the name of the col holding speed stimulus values, the name of the col containing spatial frequency,
              the name of the col holding temporal frequency, & the name of the col holding the response

  Returns the df with three extra columns holding the resp*stim for each feature

  """


  df_in = df.copy()


  #multiply the response by the three directional features
  df['stim4_feature'] = df[response] * df[stim_speed]

  df['stim7_feature'] = df[response] * df[stim_spatial]

  df['stim8_feature'] = df[response] * df[stim_temp]

  return(df)

In [ ]:
## direction_orientation_stim_features(raw_dfs[raw_fnames[0]], 'stim2', 'stim3', 'stim5', 'resp')

# RUN FUNCTIONS ON DATA

In [ ]:
# get all raw DF's
from os import listdir
root = 'drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded_CNMF/Extracted_Features/'
raw_fnames = listdir(root + 'pre-feature extract/')
raw_fnames = [x for x in raw_fnames if 'RAW_extracted_data' in x]
raw_fnames

In [ ]:
# Dictionary of df's. One for each video. You can call each df my name
raw_dfs = {}
for p in raw_fnames:
  df = pd.read_csv(root + 'pre-feature extract/' + p, index_col=['stack','label','roi','frame'])
  if 'Unnamed: 0' in df.columns:
    df = df.drop(columns='Unnamed: 0')
  raw_dfs[p] = df
raw_dfs[raw_fnames[0]]

In [ ]:
raw_dfs[raw_fnames[1]]

In [ ]:
raw_dfs.keys()

## For each df, run through pipeline

In [ ]:
#Set parameters here

response = 'resp'
time_window = 5

#declare if we're chunking the stacks into smaller segments
chunk_stacks = True
#if we are, declare how many chunks we're breaking each stack into
periods_per_stack = 10

#declare if we're only keeping positive response values or if we're using all
#if we're removing them, we get the 4-length stim1 feature, otherwise it's the 16-length stim1
only_keep_positive_resp = True



#loop through the different movies' df's and run the feature extraction functions on each
for p in raw_fnames:
  df = raw_dfs[p]

  #extract directional and orientation features
  df = direction_orientation_stim_features(df, stim_x = 'stim2', stim_y = 'stim3', stim_orientation = 'stim5', response = response)



  #ADD OTHER FEATURE EXTRACTION FUNCTIONS HERE before the contrast ones remove some rows and potentially throw things off

  #extract speed and frequency features
  df = speed_freq_Response(df, stim_speed = 'stim4', stim_spatial = 'stim7', stim_temp = 'stim8', response = response)



  #check if we're keeping negative response, then process the two contrast stimuli with whatever the choice is
  #each of these approaches will remove the final x frames from each stack, where x is the time_window declared above
  if only_keep_positive_resp == True:
    df = contrast_4_len_representation(df, 'stim1', response, time_window, zero_point = 0)
    df = contrast_4_len_representation(df, 'stim6', response, time_window, zero_point = 0)
    #remove the last frames in each stack that didn't have a future frame to look into
    df = df.dropna()
  else:
    df = contrast_16_len_representation(df, 'stim1', response, time_window, zero_point = 0)
    df = contrast_16_len_representation(df, 'stim6', response, time_window, zero_point = 0)
    #remove the last frames in each stack that didn't have a future frame to look into
    df = df.dropna()

  #if we're removing negative response frames, remove them here
  if only_keep_positive_resp == True:
    df = df.loc[df['resp'] > 0]


  #if we're chunking stacks into smaller segments, run groupFrames to assign frames to a group
  if chunk_stacks == True:
    df = groupFrames(df, periods_per_stack)

  
  raw_dfs[p] = df

In [ ]:
#look at one of the df's to see if all the columns are there
print(raw_dfs[raw_fnames[0]].columns)
raw_dfs[raw_fnames[0]]

## assign DF's with train/test labels

In [ ]:
# # SET THESE VALUES EACH TIME

# folder_name = '5_frames_stim1' # create a new folder name for saving out these files (make sure this folder already exists)
# num_train = 3
# num_test = 2
# num_val = 0

In [ ]:
# arr = np.array(['train'] * num_train + ['test'] * num_test + ['val'] * num_val)
# np.random.shuffle(arr)

# # save out each DF with a label indicating train, test or validation
# for i, p in enumerate(raw_fnames):
#   p_split = p.split('.')[-2] # chop off csv
  
#   raw_dfs[p].to_csv(root + folder_name + '/' + p_split + '_' + arr[i] + '.csv') #saves to new folder

## Save data - assign the folder manually, probably label the csv's based on the settings we used

In [ ]:
#skipped train/test because it was giving me trouble

folder_name = '5_frames_stim1'

for i, p in enumerate(raw_fnames):
  p_split = p.split('.')[-2] # chop off csv
  
  raw_dfs[p].to_csv(root + folder_name + '/' + p_split + '_features.csv') #saves to new folder

In [ ]:
listdir(root + folder_name)